In [2]:
import openai
import os

In [9]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
class Agent:
    def __init__(self, system= ""):
        self.system = system
        self.messages = []
        self.model = "gpt-4o-mini-2024-07-18"
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "system", "content": result})
        return result

    def execute(self):
        response = openai.chat.completions.create(
            model=self.model,
            temperature=0.0,
            messages=self.messages
        )
        return response.choices[0].message.content

In [26]:
prompt = """
You run in a loop of Thougth, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thougth to descrive your thougths about the question you have to answer.
Use Action to run one of the actions you have available - then return PUASE.
Observation is the result of the action you took.

Your avaliable actions are:

calculate:
e.g. calculate 4 * 7 / 3
Runs a calcuation and returns the number - uses Python so be sure to use the correct syntax.

planet_mass:
e.g. planet_mass: Earth
Returns the mass of the planet in the solar system you asked for.

Example session:

Question: What is the combined mass of Earth and Mars?
Thought: I should find the mass of Earth and Mars using the planet_mass action.
Action: planet_mass: Earth
PAUSE

You will be called again with this:

Observation: Earth has a mass of 5.972 x 10^24 kg

You then output:

Answer: Earth has a mass of 5.972 x 10^24 kg

Next, call the agent again with the following:

Action: planet_mass: Mars
PAUSE

Observation: Mars has a mass of 6.39 x 10^23 kg

you then output:

Answer: Mars has a mass of 6.39 x 10^23 kg

Finally, calculate the combined mass of Earth and Mars:

Action: calculate: 5.972 x 10^24 + 6.39 x 10^23
PAUSE

Observation: The combined mass of Earth and Mars is 6.611 x 10^24 kg

Answer: The combined mass of Earth and Mars is 6.611 x 10^24 kg.
""".strip()

In [18]:
def calculate(what):
    return eval(what)

def planet_mass(planet):
    masses = {
        "Mercury": 3.285e23,
        "Venus": 4.867e24,
        "Earth": 5.972e24,
        "Mars": 6.39e23,
        "Jupiter": 1.898e27,
        "Saturn": 5.683e26,
        "Uranus": 8.681e25,
        "Neptune": 1.024e26
    }
    return f"{planet} has a mass of {masses[planet]} x 10^24 kg"

known_actions = {
    "calculate": calculate,
    "planet_mass": planet_mass
}

In [30]:
import re
action_re = re.compile(r"^Action: (\w+): (.*)$")

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(line) for line in result.split("\n") if action_re.match(line)]
        if actions:
            action, actions_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action} : {actions_input}")
            print(f" -- Running Action: {action} : {actions_input}")
            observation = known_actions[action](actions_input)
            print('Observation:', observation)
            next_prompt = f"Observation: {observation}"
        else:
            break

In [35]:
question = "Compare the combine masses of all planets but jupiter with the mass of jupiter"
query(question, 20)

Thought: I need to find the masses of all the planets except Jupiter and then calculate their combined mass. After that, I will compare this combined mass with the mass of Jupiter. I will start by retrieving the masses of the planets one by one, excluding Jupiter.

Action: planet_mass: Mercury
PAUSE
 -- Running Action: planet_mass : Mercury
Observation: Mercury has a mass of 3.285e+23 x 10^24 kg
Answer: Mercury has a mass of 3.285e+23 kg. 

Next, I will retrieve the mass of Venus. 

Action: planet_mass: Venus
PAUSE
 -- Running Action: planet_mass : Venus
Observation: Venus has a mass of 4.867e+24 x 10^24 kg
Answer: Venus has a mass of 4.867e+24 kg.

Next, I will retrieve the mass of Earth.

Action: planet_mass: Earth
PAUSE
 -- Running Action: planet_mass : Earth
Observation: Earth has a mass of 5.972e+24 x 10^24 kg
Answer: Earth has a mass of 5.972e+24 kg.

Next, I will retrieve the mass of Mars.

Action: planet_mass: Mars
PAUSE
 -- Running Action: planet_mass : Mars
Observation: Mars 

In [36]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [37]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [38]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [39]:
graph_builder.add_edge(START, "chatbot")

In [40]:
graph_builder.add_edge("chatbot", END)

In [41]:
graph = graph_builder.compile()

In [44]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)
    # This requires some extra dependencies and is optional
    pass

HTTPSConnectionPool(host='mermaid.ink', port=443): Max retries exceeded with url: /img/JSV7aW5pdDogeydmbG93Y2hhcnQnOiB7J2N1cnZlJzogJ2xpbmVhcid9fX0lJQpncmFwaCBURDsKCV9fc3RhcnRfXyhbPHA+X19zdGFydF9fPC9wPl0pOjo6Zmlyc3QKCWNoYXRib3QoY2hhdGJvdCkKCV9fZW5kX18oWzxwPl9fZW5kX188L3A+XSk6OjpsYXN0CglfX3N0YXJ0X18gLS0+IGNoYXRib3Q7CgljaGF0Ym90IC0tPiBfX2VuZF9fOwoJY2xhc3NEZWYgZGVmYXVsdCBmaWxsOiNmMmYwZmYsbGluZS1oZWlnaHQ6MS4yCgljbGFzc0RlZiBmaXJzdCBmaWxsLW9wYWNpdHk6MAoJY2xhc3NEZWYgbGFzdCBmaWxsOiNiZmI2ZmMK?type=png&bgColor=!white (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E1ACC93590>, 'Connection to mermaid.ink timed out. (connect timeout=10)'))


In [43]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: Hello! How can I assist you today?
Assistant: As of my last knowledge update in October 2023, LangGraph is a tool designed to enhance the capabilities of large language models (LLMs) by integrating them with graph databases. It allows users to create, manage, and query knowledge graphs within the context of natural language processing. 

LangGraph can facilitate tasks such as:

1. **Knowledge Integration**: By linking structured information from graph databases with unstructured text, LangGraph can help LLMs access and utilize a richer context for generating responses or performing reasoning tasks.

2. **Interactive Querying**: Users can interact with the graph to refine their queries or gather specific information, which the language model can then process in a more informed manner.

3. **Enhanced Contextual Understanding**: By incorporating graph structures, LangGraph enables LLMs to maintain and utilize relationships and hierarchies between different data points, enhancin